# Handling Model Parameters with ParamTools
### [Jason DeBacker](https://jasondebacker.com), October 2024

## Introduction

When working with structural models, how do we handle parameter inputs?

* Worst case: hard-coded parameters in the model code
* Better: parameter values declared near the top of the main execution script
* Best: parameter values stored in a separate file and read in at the beginning of the script

But even with the best approach, we can still run into some issues;
* How do we structure metadata?
* How do we pass parameters around within the model?
* How do we handle parameter updates?
* How do we handle parameter validation?
* What do we do with time varying parameters?
* What is parameters are indexed to change over time?

In this notebook, we'll introduce [`ParamTools`](https://github.com/PSLmodels/ParamTools), a Python package that helps us handle these issues.  We'll use `ParamTools` to build a simple parameters object and show how it can help us manage parameters in a more efficient and robust way.



In [99]:
import paramtools
import taxcalc
import ogcore
import copy

## 1. Structuring Parameter Metadata

The JSON format is a human-readable, flexible format for storing hierarchical data.  We can use JSON to store metadata about our parameters.  For example, we can store information about the parameter's name, its type, its value, its description, and its constraints.  Here's an example of how we might structure metadata for a parameter:

```json
{
    "title": "Parameter Metadata",
    "type": "object",
    "properties": {
        "param_name": {
            "title": "Parameter Name",
            "type": "float",
            "description": "This is a parameter",
            "value": 0.0,
            "validators": {
                "range": {
                    "min": 0.0,
                    "max": 1.0
                }
            }
        }
    }
}
```

### Some examples of large parameter files:

1. [Tax-Calculator](https://github.com/PSLmodels/Tax-Calculator/blob/master/taxcalc/policy_current_law.json) current law policy parameters
2. [Cost-of-Capital-Calculator](http://ccc.pslmodels.org/content/intro.html), a) [current law defaults for tax rates and other non-asset-specific parameters](https://github.com/PSLmodels/Cost-of-Capital-Calculator/blob/master/ccc/default_parameters.json), b) [asset-specific depreciation parameters](https://github.com/PSLmodels/Cost-of-Capital-Calculator/blob/master/ccc/tax_depreciation_rules.json)
3. [OG-Core](https://github.com/PSLmodels/OG-Core/blob/master/ogcore/default_parameters.json) default parameters

This format of the JSON file, with meta data together with values, makes it easy to generate documentation of the model parameter.  E.g., [Tax-Calculator parameters](https://taxcalc.pslmodels.org/guide/policy_params.html), [OG-PSL parameters](https://eapd-drb.github.io/OG-PHL/content/calibration/exogenous_parameters.html)

## 2. Passing Parameters within a model

`ParamTools` allows us to create a parameters class object that can be passed around within a model.  This object can be used to store parameter values, metadata, and other information. 

This becomes extremely useful when we have a large model with many parameters that need to be passed around to different, often deeply nested, function calls.

The OG-Core model has good examples of this, e.g., in the [module that solves the model's steady-state](https://github.com/PSLmodels/OG-Core/blob/master/ogcore/SS.py).

3. Updating Parameters

`ParamTools` allows us to update parameters in a systematic way.  We can update parameters by passing a dictionary of new values (or a JSON of new values) to the `adjust` method of the ParamTools `Parameters` object (Note, this method is sometimes renamed in different implementations, e.g., in the Tax-Calculator model it is named `implement_reform`, but retains the same basic functionality).  This method will update the parameter values and validate the new values against the parameter constraints.

An example from the Tax-Calculator model:

In [87]:
from taxcalc import Policy
pol = Policy()
# view the top tax rate in 2026 under current law
pol.set_year(2026)
print(f"Top marginal IIT rate in 2026 under current law is {pol.II_rt7[0]}")


Top marginal IIT rate in 2026 under current law is 0.396


In [88]:
# update from current law policy, to full TCJA extension
json_url = 'https://raw.githubusercontent.com/PSLmodels/Tax-Calculator/master/taxcalc/reforms/ext.json'
pol.implement_reform(taxcalc.Policy.read_json_reform(json_url))
# view the top tax rate in 2026 under TCJA extension
pol.set_year(2026)
print(f"Top marginal IIT rate in 2026 under TCJA extension would be {pol.II_rt7[0]}")

Top marginal IIT rate in 2026 under TCJA extension would be 0.37


An example from the set of [OG-Core](https://github.com/PSLmodels/OG-Core)-related models:

In [110]:

# Load the default parameters for OG model
og_params = ogcore.parameters.Specifications()
# Update to USA
# copy params
usa_params = copy.deepcopy(og_params)
usa_url = 'https://raw.githubusercontent.com/PSLmodels/OG-USA/master/ogusa/ogusa_default_parameters.json'
usa_params.update_specifications(usa_url)
# Update to PHL
# copy params
zaf_params = copy.deepcopy(og_params)
zaf_url = 'https://raw.githubusercontent.com/EAPD-DRB/OG-ZAF/refs/heads/main/ogzaf/ogzaf_default_parameters.json'
zaf_params.update_specifications(zaf_url)

print(f"Debt to GDP in 2024 in the USA is {usa_params.initial_debt_ratio}")
print(f"Debt to GDP in 2024 in ZAF is {zaf_params.initial_debt_ratio}")

print(f"The corporate income tax rate in 2024 in the USA is {usa_params.cit_rate[0][0]}")
print(f"The corporate income tax rate in 2024 in ZAF is {zaf_params.cit_rate[0][0]}")


Debt to GDP in 2024 in the USA is 0.99
Debt to GDP in 2024 in ZAF is 0.74
The corporate income tax rate in 2024 in the USA is 0.21
The corporate income tax rate in 2024 in ZAF is 0.27


## 3. Parameter validation

`ParamTools` allows us to validate parameters.  When we update parameters, the `adjust` method will validate the new values against the parameter validators.  If the new values do not meet the constraints, the method will raise an error.

In [111]:
# Example of setting a float value outside of the range
usa_params.update_specifications({'cit_rate': [[1.5]]})

ValidationError: {
    "errors": {
        "cit_rate": [
            "cit_rate [[1.5]] > max 0.99 "
        ]
    }
}

In [112]:
# Example of setting a string outside a list of acceptable values
usa_params.update_specifications({'tax_func_type': "whatever"})

ValidationError: {
    "errors": {
        "tax_func_type": [
            "tax_func_type \"whatever\" must be in list of choices DEP, DEP_totalinc, GS, HSV, linear, mono, mono2D."
        ]
    }
}

## 4. Time varying and indexed parameters

In [2]:
# Define a custom TaxParams class that extends the built-in Parameters class.
class TaxParams(paramtools.Parameters):
    defaults = {
        "schema": {
            "labels": {
                "year": {
                    "type": "int",
                    "validators": {"range": {"min": 2013, "max": 2033}}
                },
                "marital_status": {
                    "type": "str",
                    "validators": {"choice": {"choices": ["single", "joint"]}}
                },
            }
        },
        "standard_deduction": {
            "title": "Standard deduction amount",
            "description": "Amount filing unit can use as a standard deduction.",
            "type": "float",

            # Set indexed to True to extend standard_deduction with the built-in
            # extension logic.
            "indexed": True,
            "value": [
                {"year": 2013, "marital_status": "single", "value": 6100.0},
                {"year": 2013, "marital_status": "joint", "value": 12200.0},
                {"year": 2018, "marital_status": "single", "value": 12000},
                {"year": 2018, "marital_status": "joint", "value": 24000},
                {"year": 2026, "marital_status": "single", "value": 7685},
                {"year": 2026, "marital_status": "joint", "value": 15369}],
            "validators": {
                "range": {
                    "min": 0,
                    "max": 9e+99
                }
            }
        },
    }
    array_first = True
    label_to_extend = "year"
    # Activate use of extend_func method.
    uses_extend_func = True
    # inflation rates from Tax-Calculator v4.3.0
    index_rates = {
        2013: 0.0148,
        2014: 0.0159,
        2015: 0.0012,
        2016: 0.0126,
        2017: 0.0167,
        2018: 0.02,
        2019: 0.013,
        2020: 0.008,
        2021: 0.0427,
        2022: 0.0723,
        2023: 0.054,
        2024: 0.055,
        2025: 0.0212,
        2026: 0.0207,
        2027: 0.0195,
        2028: 0.0194,
        2029: 0.0197,
        2030: 0.0198,
        2031: 0.0199,
        2032: 0.020,
        2033: 0.020
    }

In [3]:
params = TaxParams()
params.standard_deduction

array([[ 6100.  , 12200.  ],
       [ 6190.28, 12380.56],
       [ 6288.71, 12577.41],
       [ 6296.26, 12592.5 ],
       [ 6375.59, 12751.17],
       [12000.  , 24000.  ],
       [12240.  , 24480.  ],
       [12399.12, 24798.24],
       [12498.31, 24996.63],
       [13031.99, 26063.99],
       [13974.2 , 27948.42],
       [14728.81, 29457.63],
       [15538.89, 31077.8 ],
       [ 7685.  , 15369.  ],
       [ 7844.08, 15687.14],
       [ 7997.04, 15993.04],
       [ 8152.18, 16303.3 ],
       [ 8312.78, 16624.48],
       [ 8477.37, 16953.64],
       [ 8646.07, 17291.02],
       [ 8818.99, 17636.84]])

In [17]:
# Or show as list of dicts
params.sel["standard_deduction"]

Values([
  {'value': 6100.0, 'year': 2013, 'marital_status': 'single'},
  {'value': 12200.0, 'year': 2013, 'marital_status': 'joint'},
  {'value': 12000.0, 'year': 2018, 'marital_status': 'single'},
  {'value': 24000.0, 'year': 2018, 'marital_status': 'joint'},
  {'value': 7685.0, 'year': 2026, 'marital_status': 'single'},
  {'value': 15369.0, 'year': 2026, 'marital_status': 'joint'},
  {'value': 6190.28, 'year': 2014, 'marital_status': 'single', '_auto': True},
  {'value': 6288.71, 'year': 2015, 'marital_status': 'single', '_auto': True},
  {'value': 6296.26, 'year': 2016, 'marital_status': 'single', '_auto': True},
  {'value': 6375.59, 'year': 2017, 'marital_status': 'single', '_auto': True},
  {'value': 12240.0, 'year': 2019, 'marital_status': 'single', '_auto': True},
  {'value': 12399.12, 'year': 2020, 'marital_status': 'single', '_auto': True},
  {'value': 12498.31, 'year': 2021, 'marital_status': 'single', '_auto': True},
  {'value': 13031.99, 'year': 2022, 'marital_status': 'si